In [11]:
import pandas as pd
df = pd.read_csv("IMDB_Dataset.csv")

In [14]:
df = df[["review","sentiment"]]

In [15]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [16]:
df["sentiment"].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [17]:
sentiment_label = df.sentiment.factorize()

In [18]:
text = df.review.values

In [20]:
# text

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text)

2022-05-25 13:55:01.190407: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 13:55:01.190422: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [22]:
encoded_docs = tokenizer.texts_to_sequences(text)

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size = len(tokenizer.word_index) + 1

embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           3976096   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 3,992,747
Trainable params: 3,992,747
Non-trainable params: 0
____________________________________________

In [32]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=40, batch_size=32)

Epoch 1/40
1250/1250 [==============================] - 114s 91ms/step - loss: 0.2266 - accuracy: 0.9104 - val_loss: 0.3478 - val_accuracy: 0.8768
Epoch 2/40
1250/1250 [==============================] - 109s 87ms/step - loss: 0.2184 - accuracy: 0.9141 - val_loss: 0.3228 - val_accuracy: 0.8856
Epoch 3/40
1250/1250 [==============================] - 110s 88ms/step - loss: 0.2054 - accuracy: 0.9190 - val_loss: 0.3189 - val_accuracy: 0.8854
Epoch 4/40
1250/1250 [==============================] - 110s 88ms/step - loss: 0.2033 - accuracy: 0.9208 - val_loss: 0.3462 - val_accuracy: 0.8832
Epoch 5/40
1250/1250 [==============================] - 109s 87ms/step - loss: 0.1933 - accuracy: 0.9253 - val_loss: 0.2883 - val_accuracy: 0.8898
Epoch 6/40
1250/1250 [==============================] - 109s 87ms/step - loss: 0.1907 - accuracy: 0.9265 - val_loss: 0.2943 - val_accuracy: 0.8903
Epoch 7/40
1250/1250 [==============================] - 109s 87ms/step - loss: 0.1775 - accuracy: 0.9301 - val_loss: 0

In [43]:
import pickle

# save the model to disk
pickle.dump(model, open('imdb_model.sav', 'wb'))
pickle.dump(tokenizer, open('imdb_model_tokenizer.sav', 'wb'))

2022-05-25 16:19:46.430963: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://90b7d6c9-1599-438f-ae54-2d9f2d0c6a90/assets


In [46]:
loaded_model = pickle.load(open('imdb_model.sav', 'rb'))
loaded_tokenizer = pickle.load(open('imdb_model_tokenizer.sav', 'rb'))

text = ["This is a Good news"]

tw = loaded_tokenizer.texts_to_sequences(text)
tw = pad_sequences(tw,maxlen=200)
prediction = int(loaded_model.predict(tw).round().item())
print(prediction)
print("Predicted label: ", sentiment_label[1][prediction])

0
Predicted label:  positive
